In [1]:
# Importing essential libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn 
import imblearn 
import itertools
import os as os
import pickle
import random
import cv2
import time 
import PIL
from PIL import Image
import tensorflow as tf
import keras 

import tensorflow.keras.backend as K
from keras.optimizers import Adam, SGD, RMSprop,Adagrad,Adadelta,Adamax,Nadam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from keras.layers import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.utils import multi_gpu_model
from keras.callbacks import EarlyStopping
from keras.regularizers import l1, l2

from sklearn.metrics import confusion_matrix, recall_score, classification_report,\
                            roc_auc_score, roc_curve, precision_score,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from IPython.display import display 


ModuleNotFoundError: No module named 'cv2'

In [2]:
pickle_off = open('your data path',"rb")
data = pickle.load(pickle_off)
random.shuffle(data)
X=[]
y=[]
for ins in data:
    X.append(ins[0])

    y+=ins[1]
img_size=640

for x in range(len(X)):
    X[x]=cv2.resize(X[x],(img_size,img_size))
    
X = np.array(X).reshape(-1, img_size, img_size,1)
y=np.array(y)

In [4]:
# Preparing the data augumentation generator specifications: 
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1/255
    )

In [3]:
# Calculate and assign classs weights to a dictionary:
class_weights = sklearn.utils.class_weight.compute_class_weight('balanced',
                                                 np.unique(y),
                                                 y)
class_weight={}
class_weight[0]=class_weights[0]
class_weight[1]=class_weights[1]

NameError: name 'y' is not defined

In [7]:
# Independent performance functions 
def perf_measure(y_actual, y_hat, ypredic_per):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==0:
            TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN += 1

    precision = precision_score(y_actual, y_hat)
    recall = recall_score(y_actual, y_hat)
    accuracy = accuracy_score(y_actual, y_hat)
    f1 = f1_score(y_actual, y_hat)
    auc_keras=roc_auc_score(y_actual,ypredic_per)
    
    return(TP, FP, TN, FN,precision,recall,accuracy,f1,auc_keras )




In [8]:
X=X
Y = y

                         


In [9]:
# Prepare the 5 fold cross validation function specification:
skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 
# This table will contain the performance of each fold of the cross validation 
final_table=pd.DataFrame(columns=['TP', 'FP', 'TN', 'FN','precision','recall','accuracy','f1'])
# The directory where the models are going to be saved
save_dir = '/saved_models/'
# fold counter
fold_var = 1


for train_index, val_index in skf.split(X,Y):
    K.clear_session()    
    #Trainig and validation assigniment
    trainingX=X[train_index]
    trainingY=Y[train_index]
    
    valX=X[val_index]
    valY=Y[val_index]
    
    K.clear_session()
    # Creating the model, this wil be explained in the next step
    
    model = Sequential()
    '''Model structure '''
    
    opt=keras.optimizers.Adam(lr=0.00003, beta_1=0.9, beta_2=0.999, amsgrad=False, decay=1e-6)

    
    #compile the model
    
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['val_loss'])
    #callback
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100,restore_best_weights=True )

    # train the K fold with data augmentation generator
    model.fit(datagen.flow(trainingX,trainingY, batch_size=5),callbacks=[es],\
              class_weight=class_weight,verbose=0,epochs=1500 ,validation_data=(valX,valY))

    # Predict on the out of boot (validation)
    result = model.evaluate(valX,valY)
    results = dict(zip(model.metrics_names,result))
    ypredic_per=model.predict(valX)

    yvalpred=model.predict_classes(valX)
    print(perf_measure(valY,yvalpred,ypredic_per))
    pd.concat([final_table,perf_measure(valY,yvalpred,ypredic_per)], axis=0)
    
    model.save(save_dir+str(fold_var)+".h5")
    
    
    
    
    print( 'model {} accuracy is {} and the loss is {}'.format(fold_var,results['accuracy'], results['loss']))
    K.clear_session()
    fold_var += 1


C:\anaconda3\envs\TFforDAI\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\anaconda3\envs\TFforDAI\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at C:\anaconda3\envs\TFforDAI\lib\site-packages\keras\backend\tensorflow_backend.py:3007) ]] [Op:__inference_keras_scratch_graph_2107]

Function call stack:
keras_scratch_graph


In [ ]:
trial=pd.DataFrame(columns=['TP', 'FP', 'TN', 'FN','precision','recall','accuracy','f1'])

In [ ]:
trial.append(pd.Series(list(ll), index=['TP', 'FP', 'TN', 'FN','precision','recall','accuracy','f1']), ignore_index=True)

In [ ]:
trial2=pd.DataFrame(columns=['TP', 'FP', 'TN', 'FN','precision','recall','accuracy','f1'])

In [ ]:
trial2.append(pd.Series(list(ll2), index=['TP', 'FP', 'TN', 'FN','precision','recall','accuracy','f1']), ignore_index=True)

In [ ]:
pd.concat([trial,trial2])

In [ ]:
trial

In [ ]:
ll2

In [ ]:
len(val_index)